In [35]:
# 1. การหาสมการ Forward Kinematic และการแทนค่าตัวแปรที่ติดไว้เพื่อหา End Effector

import sympy as sp

# กำหนดสัญลักษณ์เชิงสัญลักษณ์
L1, L2, L3, L4, L5, q1, q2, q3 = sp.symbols('L1 L2 L3 L4 L5 q1 q2 q3')

# กำหนดค่าคงที่
L1_val = 0.3
L2_val = 0.1
L3_val = 0.2
L4_val = 0.1
L5_val = 0.2

# Lb และ La
Lb = sp.sqrt(L2_val**2 + L4_val**2)
La = sp.sqrt(L3_val**2 + L5_val**2)

# Tranformation Matrix
T_0_1 = sp.Matrix([
    [sp.cos(q1), -sp.sin(q1), 0, 0],
    [sp.sin(q1), sp.cos(q1), 0, 0],
    [0, 0, 1, L1_val],
    [0, 0, 0, 1]
])

T_1_2 = sp.Matrix([
    [sp.cos(q2), -sp.sin(q2), 0, 0],
    [0, 0, -1, 0],
    [sp.sin(q2), sp.cos(q2), 0, 0],
    [0, 0, 0, 1]
])

T_2_3 = sp.Matrix([
    [sp.cos(q3), -sp.sin(q3), 0, Lb],
    [sp.sin(q3), sp.cos(q3), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

T_3_4 = sp.Matrix([
    [1, 0, 0, La],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# ลิสต์ของตัวแปรมุม q1, q2, q3 ทั้งหมด 10 ชุด (หน่วยเป็นเรเดียน)
angle_list = [
    (sp.pi * 25 / 180, sp.pi * 30 / 180, sp.pi * 45 / 180),
    (sp.pi * 30 / 180, sp.pi * 45 / 180, sp.pi * 60 / 180),
    (sp.pi * 10 / 180, sp.pi * 20 / 180, sp.pi * 30 / 180),
    (sp.pi * 10 / 180, sp.pi * 50 / 180, sp.pi * 30 / 180),
    (sp.pi * 20 / 180, sp.pi * 30 / 180, sp.pi * 25 / 180),
    (sp.pi * 75 / 180, sp.pi * 30 / 180, sp.pi * 50 / 180),
    (sp.pi * 60 / 180, sp.pi * 20 / 180, sp.pi * 75 / 180),
    (sp.pi * 50 / 180, sp.pi * 20 / 180, sp.pi * 60 / 180),
    (sp.pi * 20 / 180, sp.pi * 25 / 180, sp.pi * 30 / 180),
    (sp.pi * 30 / 180, sp.pi * 30 / 180, sp.pi * 45 / 180)
]

# ประกาศลิสต์สำหรับเก็บค่า x, y, z
x_vals = []
y_vals = []
z_vals = []

# ทำการคำนวณและแทนค่าในแต่ละเซตของมุม
for angles in angle_list:
    q1_val, q2_val, q3_val = angles

    # คำนวณเมทริกซ์ทั้งหมดโดยแทนค่า q1, q2, q3
    P_tt = T_0_1.subs(q1, q1_val) * T_1_2.subs(q2, q2_val) * T_2_3.subs(q3, q3_val) * T_3_4

    # ทำการ simplify เมทริกซ์
    T_total_simplified = sp.simplify(P_tt)

    # แปลงเป็นทศนิยม
    P_total_numeric = T_total_simplified.evalf()

    # ดึงค่าในคอลัมน์ 4 ของแถว 1, 2, 3 (x, y, z ตามลำดับ)
    x_vals.append(P_total_numeric[0, 3])
    y_vals.append(P_total_numeric[1, 3])
    z_vals.append(P_total_numeric[2, 3])

    # แสดงผลลัพธ์
    print(f"\nผลลัพธ์เมทริกซ์สำหรับ q1 = {q1_val}, q2 = {q2_val}, q3 = {q3_val}:")
    sp.pprint(P_total_numeric, num_columns=150)
print(f"x: {x_vals}")
print(f"y: {y_vals}")
print(f"z: {z_vals}")


ผลลัพธ์เมทริกซ์สำหรับ q1 = 5*pi/36, q2 = pi/6, q3 = pi/4:
⎡0.234569716009804  -0.875426098065593  0.422618261740699  0.177345916148154 ⎤
⎢                                                                            ⎥
⎢0.109381654946615  -0.408217893676735  -0.90630778703665  0.0826977588423984⎥
⎢                                                                            ⎥
⎢0.965925826289068  0.258819045102521           0          0.643915758875543 ⎥
⎢                                                                            ⎥
⎣        0                  0                   0                 1.0        ⎦

ผลลัพธ์เมทริกซ์สำหรับ q1 = pi/6, q2 = pi/4, q3 = pi/3:
⎡-0.224143868042013  -0.836516303737808         0.5          0.0232050807568877⎤
⎢                                                                              ⎥
⎢-0.12940952255126   -0.482962913144534  -0.866025403784439  0.0133974596215561⎥
⎢                                                                              ⎥
⎢0.96592

In [38]:
# 2.เขียนสมการ Inverse Kinematic ที่คำนวณได้โดยแทนค่า End Effector ที่หามาได้เพื่อหาค่ามุม q
from prettytable import PrettyTable
import sympy as sp

# กำหนดค่าคงที่
L1 = 0.3
L2 = 0.1
L3 = 0.2
L4 = 0.1
L5 = 0.2

# คำนวณ La และ Lb
La = sp.sqrt(L2**2 + L4**2)
Lb = sp.sqrt(L3**2 + L5**2)

# สร้างตารางใหม่ที่รวมคอลัมน์จากทั้ง table และ table1
combined_table = PrettyTable()
combined_table.field_names = ["q1", "q2", "q3", "q1_1", "q1_2", "q2_1", "q2_2", "q3_1", "q3_2"]

# สร้าง array เพื่อเก็บค่ามุม
q1_1_array = []
q1_2_array = []
q2_1_array = []
q2_2_array = []
q3_1_array = []
q3_2_array = []

# ลูปคำนวณมุม q1, q2, q3 และเพิ่มข้อมูลลงในตาราง
#จากในข้อสอบกำหนด Py = z , Px = x
for i in range(len(x_vals)):
    x = x_vals[i]
    y = y_vals[i]
    z = z_vals[i]

    # ดึงค่าจาก angle_list
    q1, q2, q3 = angle_list[i]

    # r 
    r = sp.sqrt(x**2 + y**2)


    # คำนวณ q3 โดยแทนค่า S3 เป็นบวกและลบจึงได้คำตอบออกมา 2 คำตอบ
    C3 = (r**2 - La**2 - Lb**2) / (2 * La * Lb)
    S3_1 = sp.sqrt(1 + C3**2)
    S3_2 = -(sp.sqrt(1 + C3**2))


    # คำนวณ q1 ทำการบวก pi เพื่อให้มีทิศการหันหน้าและหันหลัง
    q1_1 = sp.atan2(z, x)
    q1_2 = sp.pi + sp.atan2(z, x)

    # คำนวณ q2 
    zeta1 = sp.atan2(Lb*S3_1,La+(Lb*C3))
    zeta2 = sp.atan2(Lb*S3_2,La+(Lb*C3))
    q2_1 = sp.atan2(L2+L3,L4+L5)-zeta1
    q2_2 = sp.atan2(L2+L3,L4+L5)-zeta2

    # คำนวณ q3 โดยแทนค่า sin เป็นบวกและลบจึงได้คำตอบออกมา 2 คำตอบ
    q3_1 = sp.atan2(S3_1,C3)
    q3_2 = sp.atan2(S3_2,C3)

    # เก็บค่าลงใน array
    q1_1_array.append(q1_1.evalf())
    q1_2_array.append(q1_2.evalf())
    q2_1_array.append(q2_1.evalf())
    q2_2_array.append(q2_2.evalf())
    q3_1_array.append(q3_1.evalf())
    q3_2_array.append(q3_2.evalf())

    # เพิ่มข้อมูลมุม q1, q2, q3 และค่าที่คำนวณได้ลงในตาราง
    combined_table.add_row([q1.evalf(), q2.evalf(), q3.evalf(), q1_1.evalf(), q1_2.evalf(), q2_1.evalf(), q2_2.evalf(), q3_1.evalf(), q3_2.evalf()])

# แสดงผลลัพธ์
print(combined_table)

+-------------------+-------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+
|         q1        |         q2        |         q3        |       q1_1       |       q1_2       |        q2_1        |       q2_2       |       q3_1       |        q3_2       |
+-------------------+-------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+
| 0.436332312998582 | 0.523598775598299 | 0.785398163397448 | 1.30204165345695 | 4.44363430704674 | -0.997051244283173 | 2.56784757107807 | 2.11910009976660 | -2.11910009976660 |
| 0.523598775598299 | 0.785398163397448 |  1.04719755119660 | 1.53634041196122 | 4.67793306555101 | -1.22061267486885  | 2.79140900166374 | 2.23239300804143 | -2.23239300804143 |
| 0.174532925199433 | 0.349065850398866 | 0.523598775598299 | 1.06911163161477 | 4.21070428520457 | -0.33

In [37]:
# 3.ทำการแทนค่ามุม q ทั้งหมดที่หามาได้ในสมการ Forwrad Kinematic เพื่อตรวจสอบความถูกต้องอีก 1 ครั้ง
import sympy as sp

# Define symbolic variables
L1, L2, L3, L4, L5, q1, q2, q3 = sp.symbols('L1 L2 L3 L4 L5 q1 q2 q3')

# Define constant values
L1_val = 0.3
L2_val = 0.1
L3_val = 0.2
L4_val = 0.1
L5_val = 0.2

# Define Lb and La
Lb = sp.sqrt(L2_val**2 + L4_val**2)
La = sp.sqrt(L3_val**2 + L5_val**2)

# Define the transformation matrices
T_0_1 = sp.Matrix([
    [sp.cos(q1), -sp.sin(q1), 0, 0],
    [sp.sin(q1), sp.cos(q1), 0, 0],
    [0, 0, 1, L1_val],
    [0, 0, 0, 1]
])

T_1_2 = sp.Matrix([
    [sp.cos(q2), -sp.sin(q2), 0, 0],
    [0, 0, -1, 0],
    [sp.sin(q2), sp.cos(q2), 0, 0],
    [0, 0, 0, 1]
])

T_2_3 = sp.Matrix([
    [sp.cos(q3), -sp.sin(q3), 0, Lb],
    [sp.sin(q3), sp.cos(q3), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

T_3_4 = sp.Matrix([
    [1, 0, 0, La],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Arrays of q values
q1_1_array = [0.436332312998582, 0.523598775598299, 0.174532925199433, 0.174532925199433, 0.349065850398866, 
              1.30899693899575, 1.04719755119660, 0.872664625997165, 0.349065850398866, 0.523598775598299]

q1_2_array = [3.57792496658838, 3.66519142918809, 3.31612557878923, 3.31612557878923, 3.49065850398866, 
              4.45058959258554, 4.18879020478639, 4.01425727958696, 3.49065850398866, 3.66519142918809]

q2_1_array = [0.523598775598298, 0.785398163397448, 0.349065850398865, 0.872664625997164, 0.523598775598298, 
              0.523598775598299, 0.349065850398866, 0.349065850398866, 0.436332312998583, 0.523598775598299]

q2_2_array = [1.58340435509615, 2.21284692128698, 1.05082467214338, 1.57442344774168, 1.10745937226943, 
              1.70466726477960, 2.15887709004555, 1.77651460828840, 1.13809113474310, 1.58340435509615]

q3_1_array = [0.785398163397448, 1.04719755119660, 0.523598775598299, 0.523598775598299, 0.436332312998583, 
              0.872664625997165, 1.30899693899575, 1.04719755119660, 0.523598775598298, 0.785398163397448]

q3_2_array = [5.49778714378214, 5.23598775598299, 5.75958653158129, 5.75958653158129, 5.84685299418100, 
              5.41052068118242, 4.97418836818384, 5.23598775598299, 5.75958653158129, 5.49778714378214]

# Function to compute the total transformation matrix for given q1, q2, q3
def compute_transformation(q1_val, q2_val, q3_val):
    T_total = T_0_1.subs(q1, q1_val) * T_1_2.subs(q2, q2_val) * T_2_3.subs(q3, q3_val) * T_3_4
    return sp.simplify(T_total)

# Initialize arrays for storing results
X_Sol1 = []
Y_Sol1 = []
Z_Sol1 = []

X_Sol2 = []
Y_Sol2 = []
Z_Sol2 = []

X_Sol3 = []
Y_Sol3 = []
Z_Sol3 = []

X_Sol4 = []
Y_Sol4 = []
Z_Sol4 = []

# Iterate through the arrays and compute the transformation matrix for each set of angles
for i in range(len(q1_1_array)):
    # Compute transformation matrices for each set of angles
    T_numeric_1 = compute_transformation(q1_1_array[i], q2_1_array[i], q3_1_array[i])
    T_numeric_2 = compute_transformation(q1_1_array[i], q2_2_array[i], q3_2_array[i])
    T_numeric_3 = compute_transformation(q1_2_array[i], q2_1_array[i], q3_1_array[i])
    T_numeric_4 = compute_transformation(q1_2_array[i], q2_2_array[i], q3_2_array[i])
    
    # Extract the fourth column (position vector) for the first set of angles
    X_Sol1.append(T_numeric_1[0, 3])
    Y_Sol1.append(T_numeric_1[1, 3])
    Z_Sol1.append(T_numeric_1[2, 3])

    # Extract the fourth column (position vector) for the first set of angles
    X_Sol2.append(T_numeric_2[0, 3])
    Y_Sol2.append(T_numeric_2[1, 3])
    Z_Sol2.append(T_numeric_2[2, 3])

    # Extract the fourth column (position vector) for the first set of angles
    X_Sol3.append(T_numeric_3[0, 3])
    Y_Sol3.append(T_numeric_3[1, 3])
    Z_Sol3.append(T_numeric_3[2, 3])
    
    # Extract the fourth column (position vector) for the second set of angles
    X_Sol4.append(T_numeric_4[0, 3])
    Y_Sol4.append(T_numeric_4[1, 3])
    Z_Sol4.append(T_numeric_4[2, 3])
def make_positive(arr):
    return [abs(val) for val in arr]

# มุมที่ได้บางมุมเกิน Work Space ที่หุ่นทำได้ทำให้ต้องมีการกำหนด Work Space
X_Sol1 = make_positive(X_Sol1)
X_Sol2 = make_positive(X_Sol2)
X_Sol3 = make_positive(X_Sol3)
X_Sol4 = make_positive(X_Sol4)

Y_Sol1 = make_positive(Y_Sol1)
Y_Sol2 = make_positive(Y_Sol2)
Y_Sol3 = make_positive(Y_Sol3)
Y_Sol4 = make_positive(Y_Sol4)

Z_Sol1 = make_positive(Z_Sol1)
Z_Sol2 = make_positive(Z_Sol2)
Z_Sol3 = make_positive(Z_Sol3)
Z_Sol4 = make_positive(Z_Sol4)

# Display results
print("Results for q1_1, q2_1, q3_1 (positive):")
print("X_Sol1 =", X_Sol1)
print("Y_Sol1 =", Y_Sol1)
print("Z_Sol1 =", Z_Sol1)

print("\nResults for q1_1, q2_2, q3_2 (positive):")
print("X_Sol2 =", X_Sol2)
print("Y_Sol2 =", Y_Sol2)
print("Z_Sol2 =", Z_Sol2)

print("\nResults for q1_2, q2_1, q3_1 (positive):")
print("X_Sol3 =", X_Sol3)
print("Y_Sol3 =", Y_Sol3)
print("Z_Sol3 =", Z_Sol3)

print("\nResults for q1_2, q2_2, q3_2 (positive):")
print("X_Sol4 =", X_Sol4)
print("Y_Sol4 =", Y_Sol4)
print("Z_Sol4 =", Z_Sol4)
# ประกาศลิสต์สำหรับเก็บค่า x, y, z
x_vals = []
y_vals = []
z_vals = []

# ทำการคำนวณและแทนค่าในแต่ละเซตของมุม
for angles in angle_list:
    q1_val, q2_val, q3_val = angles

    # คำนวณเมทริกซ์ทั้งหมดโดยแทนค่า q1, q2, q3
    P_tt = T_0_1.subs(q1, q1_val) * T_1_2.subs(q2, q2_val) * T_2_3.subs(q3, q3_val) * T_3_4

    # ทำการ simplify เมทริกซ์
    T_total_simplified = sp.simplify(P_tt)

    # แปลงเป็นทศนิยม
    P_total_numeric = T_total_simplified.evalf()

    # ดึงค่าในคอลัมน์ 4 ของแถว 1, 2, 3 (x, y, z ตามลำดับ)
    x_vals.append(P_total_numeric[0, 3])
    y_vals.append(P_total_numeric[1, 3])
    z_vals.append(P_total_numeric[2, 3])

print('\n')
print(f"x: {x_vals}")
print(f"y: {y_vals}")
print(f"z: {z_vals}")

Results for q1_1, q2_1, q3_1 (positive):
X_Sol1 = [0.177345916148154, 0.0232050807568873, 0.309919389804177, 0.137891813630893, 0.267536505245564, 0.0444106586801716, 0.0541206190956230, 0.116992311439127, 0.272889736086634, 0.169463476799538]
Y_Sol1 = [0.0826977588423985, 0.0133974596215559, 0.0546471502671803, 0.0243140471619497, 0.0973753244889891, 0.165742834592002, 0.0937396620107019, 0.139426007463480, 0.0993237411722942, 0.0978397839480234]
Z_Sol1 = [0.643915758875542, 0.673205080756888, 0.565039040697476, 0.686880740211948, 0.602401864254482, 0.649256374246662, 0.630135363090699, 0.626914648657603, 0.591458433881852, 0.643915758875543]

Results for q1_1, q2_2, q3_2 (positive):
X_Sol2 = [0.177345916148154, 0.0232050807568874, 0.309919389804177, 0.137891813630892, 0.267536505245564, 0.0444106586801714, 0.0541206190956239, 0.116992311439126, 0.272889736086633, 0.169463476799539]
Y_Sol2 = [0.0826977588423985, 0.0133974596215560, 0.0546471502671803, 0.0243140471619496, 0.09737532448